In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import tmean
from scipy.stats.mstats import trima
import numpy as np
import dautil as dl
import seaborn as sns
from IPython.display import HTML

In [ ]:
context = dl.nb.Context('central_tendency')
lr = dl.nb.LatexRenderer(chapter=4, context=context)
lr.render(r'TM= \frac{Q_1 + 2Q_2 + Q_3}{4}')
lr.render(r'x_\mathrm{IQM} = {2 \over n} \sum_{i=\frac{n}{4}+1}^{\frac{3n}{4}}{x_i}')

In [ ]:
def iqm(a):
    return truncated_mean(a, 25)

In [ ]:
def plotdists(var, ax):
    displot_label = 'From {0} to {1} percentiles'
    cyc = dl.plotting.Cycler()

    for i in range(1, 9, 3):
        limits = dl.stats.outliers(var, method='percentiles', 
                               percentiles=(i, 100 - i))
        truncated = trima(var, limits=limits).compressed()
        sns.distplot(truncated, ax=ax, color=cyc.color(),
                     hist_kws={'histtype': 'stepfilled', 'alpha': 1/i,
                               'linewidth': cyc.lw()},
                     label=displot_label.format(i, 100 - i))

In [ ]:
def truncated_mean(a, percentile):
    limits = dl.stats.outliers(a, method='percentiles', 
                               percentiles=(percentile, 100 - percentile))
    
    return tmean(a, limits=limits)

In [ ]:
df = dl.data.Weather.load().resample('M').dropna()
x = range(9)
temp_means = [truncated_mean(df['TEMP'], i) for i in x]
ws_means = [truncated_mean(df['WIND_SPEED'], i) for i in x]

In [ ]:
%matplotlib inline
dl.nb.RcWidget(context)
dl.nb.LabelWidget(2, 2, context)

In [ ]:
sp = dl.plotting.Subplotter(2, 2, context)
cp = dl.plotting.CyclePlotter(sp.ax)
cp.plot(x, temp_means, label='Truncated mean')
cp.plot(x, dl.stats.trimean(df['TEMP']) * np.ones_like(x), label='Trimean')
cp.plot(x, iqm(df['TEMP']) * np.ones_like(x), label='IQM')
sp.label(ylabel_params=dl.data.Weather.get_header('TEMP'))

cp = dl.plotting.CyclePlotter(sp.next_ax())
cp.plot(x, ws_means, label='Truncated mean')
cp.plot(x, dl.stats.trimean(df['WIND_SPEED']) * np.ones_like(x),
        label='Trimean')
cp.plot(x, iqm(df['WIND_SPEED']) * np.ones_like(x), label='IQM')
sp.label(ylabel_params=dl.data.Weather.get_header('WIND_SPEED'))

plotdists(df['TEMP'], sp.next_ax())
sp.label(xlabel_params=dl.data.Weather.get_header('TEMP'))

plotdists(df['WIND_SPEED'], sp.next_ax())
sp.label(xlabel_params=dl.data.Weather.get_header('WIND_SPEED'))
plt.tight_layout()
HTML(dl.report.HTMLBuilder().watermark())